# scikit-learn随机森林调参小结

在[Bagging与随机森林算法原理小结中]()，我们对随机森林(Random Forest, 以下简称RF）的原理做了总结。本文就从实践的角度对RF做一个总结。重点讲述scikit-learn中RF的调参注意事项，以及和GBDT调参的异同点。

## 1. scikit-learn随机森林类库概述

在scikit-learn中，RF的分类类是RandomForestClassifier，回归类是RandomForestRegressor。当然RF的变种Extra Trees也有， 分类类ExtraTreesClassifier，回归类ExtraTreesRegressor。由于RF和Extra Trees的区别较小，调参方法基本相同，本文只关注于RF的调参。

和GBDT的调参类似，RF需要调参的参数也包括两部分，第一部分是Bagging框架的参数，第二部分是CART决策树的参数。下面我们就对这些参数做一个介绍。

## 2.  RF框架参数

首先我们关注于RF的Bagging框架的参数。这里可以和GBDT对比来学习。在[scikit-learn 梯度提升树(GBDT)调参小结](http://www.cnblogs.com/pinard/p/6143927.html)中我们对GBDT的框架参数做了介绍。GBDT的框架参数比较多，重要的有最大迭代器个数，步长和子采样比例，调参起来比较费力。但是RF则比较简单，这是因为bagging框架里的各个弱学习器之间是没有依赖关系的，这减小的调参的难度。换句话说，达到同样的调参效果，RF调参时间要比GBDT少一些。

下面我来看看RF重要的Bagging框架的参数，由于RandomForestClassifier和RandomForestRegressor参数绝大部分相同，这里会将它们一起讲，不同点会指出。

　1) **n_estimators**: 也就是弱学习器的最大迭代次数，或者说最大的弱学习器的个数。一般来说n_estimators太小，容易欠拟合，n_estimators太大，计算量会太大，并且n_estimators到一定的数量后，再增大n_estimators获得的模型提升会很小，所以一般选择一个适中的数值。默认是100。

　2) **oob_score** :即是否采用袋外样本来评估模型的好坏。默认识False。个人推荐设置为True，因为袋外分数反应了一个模型拟合后的泛化能力。

　3) **criterion**: 即CART树做划分时对特征的评价标准。分类模型和回归模型的损失函数是不一样的。分类RF对应的CART分类树默认是基尼系数gini,另一个可选择的标准是信息增益。回归RF对应的CART回归树默认是均方差mse，另一个可以选择的标准是绝对值差mae。一般来说选择默认的标准就已经很好的。

　从上面可以看出， RF重要的框架参数比较少，主要需要关注的是 n_estimators，即RF最大的决策树个数。

## 3.  RF决策树参数

　下面我们再来看RF的决策树参数，它要调参的参数基本和GBDT相同，如下:



　1) RF划分时考虑的最大特征数**max_features**: 可以使用很多种类型的值，默认是"auto",意味着划分时最多考虑$$\sqrt{N}$$个特征；如果是"log2"意味着划分时最多考虑个$$\log_2N$$特征；如果是"sqrt"或者"auto"意味着划分时最多考虑$$\sqrt{N}$$个特征。如果是整数，代表考虑的特征绝对数。如果是浮点数，代表考虑特征百分比，即考虑（百分比xN）取整后的特征数。其中N为样本总特征数。一般我们用默认的"auto"就可以了，如果特征数非常多，我们可以灵活使用刚才描述的其他取值来控制划分时考虑的最大特征数，以控制决策树的生成时间。

　2) 决策树最大深度**max_depth**: 默认可以不输入，如果不输入的话，决策树在建立子树的时候不会限制子树的深度。一般来说，数据少或者特征少的时候可以不管这个值。如果模型样本量多，特征也多的情况下，推荐限制这个最大深度，具体的取值取决于数据的分布。常用的可以取值10-100之间。

　3) 内部节点再划分所需最小样本数**min_samples_split**: 这个值限制了子树继续划分的条件，如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分。 默认是2.如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

　4) 叶子节点最少样本数**min_samples_leaf**: 这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝。 默认是1,可以输入最少的样本数的整数，或者最少样本数占样本总数的百分比。如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

　5）叶子节点最小的样本权重和**min_weight_fraction_leaf**：这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起被剪枝。 默认是0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。

　6) 最大叶子节点数**max_leaf_nodes**: 通过限制最大叶子节点数，可以防止过拟合，默认是"None”，即不限制最大的叶子节点数。如果加了限制，算法会建立在最大叶子节点数内最优的决策树。如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制，具体的值可以通过交叉验证得到。

　7) 节点划分最小不纯度**min_impurity_split**:  这个值限制了决策树的增长，如果某节点的不纯度(基于基尼系数，均方差)小于这个阈值，则该节点不再生成子节点。即为叶子节点 。一般不推荐改动默认值$$1e-7$$。

　上面决策树参数中最重要的包括最大特征数max_features， 最大深度max_depth， 内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf。

## 4.RF调参实例

　这里仍然使用GBDT调参时同样的数据集来做RF调参的实例，数据的[下载地址在这](http://files.cnblogs.com/files/pinard/train_modified.zip)。本例我们采用袋外分数来评估我们模型的好坏。

　首先，我们载入需要的类库：

In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation, metrics

import matplotlib.pylab as plt
%matplotlib inline

F:\anaconda\lib\site-packages\sklearn\cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "new CV iterators are different from that of this module. "


　接着，我们把解压的数据用下面的代码载入，顺便看看数据的类别分布。

In [6]:
train = pd.read_csv('./dataset/train_modified.csv')
target='Disbursed' # Disbursed的值就是二元分类的输出
IDcol = 'ID'
train['Disbursed'].value_counts() 

0    19680
1      320
Name: Disbursed, dtype: int64

　可以看到类别输出如下，也就是类别0的占大多数。

　接着我们选择好样本特征和类别输出。

In [7]:
x_columns = [x for x in train.columns if x not in [target, IDcol]]
X = train[x_columns]
y = train['Disbursed']

　不管任何参数，都用默认的，我们拟合下数据看看：

In [13]:
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X,y)
print (rf0.oob_score_)
y_predprob = rf0.predict_proba(X)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

F:\anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
F:\anaconda\lib\site-packages\sklearn\ensemble\forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
F:\anaconda\lib\site-packages\sklearn\ensemble\forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


0.98005
AUC Score (Train): 0.999833


　输出如下，可见袋外分数已经很高，而且AUC分数也很高。相对于GBDT的默认参数输出，RF的默认参数拟合效果对本例要好一些。

　我们首先对n_estimators进行网格搜索：

In [18]:
from sklearn.model_selection import GridSearchCV 
param_test1 = {'n_estimators':range(10,71,10)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X,y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessi

({'mean_fit_time': array([0.0754014 , 0.1294003 , 0.18580098, 0.23500047, 0.29440036,
         0.34760079, 0.40700054]),
  'std_fit_time': array([0.00937411, 0.00542606, 0.00475032, 0.0058653 , 0.00463031,
         0.00553547, 0.00750999]),
  'mean_score_time': array([0.00999999, 0.01059999, 0.01399999, 0.02240019, 0.02800012,
         0.02599993, 0.03020005]),
  'std_score_time': array([0.        , 0.00120037, 0.00489907, 0.00387819, 0.00400007,
         0.00489894, 0.00040004]),
  'param_n_estimators': masked_array(data=[10, 20, 30, 40, 50, 60, 70],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 10},
   {'n_estimators': 20},
   {'n_estimators': 30},
   {'n_estimators': 40},
   {'n_estimators': 50},
   {'n_estimators': 60},
   {'n_estimators': 70}],
  'split0_test_score': array([0.81797431, 0.82673558, 0.8370927 , 0.83676321, 0.8351753 ,
         0.83643769, 0.83286093]),
  'spl

　这样我们得到了最佳的弱学习器迭代次数，接着我们对决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split进行网格搜索。

In [19]:
param_test2 = {'max_depth':range(3,14,2), 'min_samples_split':range(50,201,20)}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, 
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessi

({'mean_fit_time': array([0.35180693, 0.34220405, 0.33940368, 0.34040365, 0.33600311,
         0.34240355, 0.3414031 , 0.33840351, 0.40640254, 0.4014039 ,
         0.40960407, 0.40760627, 0.40720396, 0.41600471, 0.43640413,
         0.40560441, 0.47080574, 0.47360516, 0.46840525, 0.46200447,
         0.46120515, 0.46460552, 0.45600471, 0.4628046 , 0.51420474,
         0.50220494, 0.50640569, 0.50940528, 0.50780435, 0.49880562,
         0.50220509, 0.49520535, 0.5314064 , 0.53160558, 0.53100581,
         0.52820525, 0.52560506, 0.51860504, 0.51680527, 0.51640549,
         0.54560704, 0.54820733, 0.53380542, 0.5302052 , 0.53080587,
         0.53540511, 0.521205  , 0.5336061 ]),
  'std_fit_time': array([0.02009792, 0.00530737, 0.00403166, 0.00492325, 0.0049806 ,
         0.00519923, 0.00656041, 0.00382633, 0.00725629, 0.00427033,
         0.0046298 , 0.00574887, 0.00312344, 0.00887777, 0.02215971,
         0.00458718, 0.00699681, 0.00516097, 0.00682995, 0.00907795,
         0.00783268, 0.

　　我们看看我们现在模型的袋外分数：

In [20]:
rf1 = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=110,
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=10)
rf1.fit(X,y)
print(rf1.oob_score_)

0.984


　可见此时我们的袋外分数有一定的提高。也就是时候模型的泛化能力增强了。

　对于内部节点再划分所需最小样本数min_samples_split，我们暂时不能一起定下来，因为这个还和决策树其他的参数存在关联。下面我们再对内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf一起调参。

In [21]:
param_test3 = {'min_samples_split':range(80,150,20), 'min_samples_leaf':range(10,60,10)}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, max_depth=13,
                                  max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(X,y)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessi

({'mean_fit_time': array([0.56800942, 0.54520688, 0.55380569, 0.58761935, 0.55300579,
         0.58561115, 0.5950192 , 0.55181093, 0.54380846, 0.52940736,
         0.52600532, 0.51880474, 0.52680664, 0.52040367, 0.5180048 ,
         0.51200509, 0.54420958, 0.51680684, 0.50800452, 0.58381062]),
  'std_fit_time': array([0.03551238, 0.00847019, 0.02451506, 0.03208686, 0.01623646,
         0.03545666, 0.03660507, 0.01390875, 0.01604697, 0.00531599,
         0.00804966, 0.00376228, 0.00818583, 0.00849945, 0.00309864,
         0.00468938, 0.05372474, 0.00890977, 0.00547792, 0.07357947]),
  'mean_score_time': array([0.03400035, 0.03220072, 0.0398006 , 0.03300114, 0.03300042,
         0.04060197, 0.03460073, 0.03600144, 0.03380079, 0.0306004 ,
         0.03080029, 0.03180032, 0.03659997, 0.02980032, 0.0318006 ,
         0.03080015, 0.03080025, 0.02880044, 0.0318007 , 0.03460097]),
  'std_score_time': array([0.00536632, 0.00391969, 0.00815911, 0.00368866, 0.00464807,
         0.00680069, 0.0034

　最后我们再对最大特征数max_features做调参:

In [22]:
param_test4 = {'max_features':range(3,11,2)}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=120,
                                  min_samples_leaf=20 ,oob_score=True, random_state=10),
   param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(X,y)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessi

({'mean_fit_time': array([0.39520717, 0.46200576, 0.53220644, 0.59480805]),
  'std_fit_time': array([0.01810722, 0.00497952, 0.00803509, 0.00949561]),
  'mean_score_time': array([0.03200002, 0.03320079, 0.03440089, 0.03540087]),
  'std_score_time': array([0.00399997, 0.00407002, 0.00338201, 0.00407898]),
  'param_max_features': masked_array(data=[3, 5, 7, 9],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_features': 3},
   {'max_features': 5},
   {'max_features': 7},
   {'max_features': 9}],
  'split0_test_score': array([0.81893102, 0.82697972, 0.83293834, 0.81775994]),
  'split1_test_score': array([0.79912387, 0.79626763, 0.79838748, 0.80414563]),
  'split2_test_score': array([0.78474935, 0.77782211, 0.80341916, 0.78332023]),
  'split3_test_score': array([0.84169565, 0.83561793, 0.8346374 , 0.83346434]),
  'split4_test_score': array([0.85455967, 0.8452466 , 0.85494276, 0.84648517]),
  'mean_test_score': array(

　用我们搜索到的最佳参数，我们再看看最终的模型拟合：

In [24]:
rf2 = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=120,
                                  min_samples_leaf=20,max_features=7 ,oob_score=True, random_state=10)
rf2.fit(X,y)
print(rf2.oob_score_)

0.984


　可见此时模型的袋外分数基本没有提高，主要原因是0.984已经是一个很高的袋外分数了，如果想进一步需要提高模型的泛化能力，我们需要更多的数据。

文章转载自：博客园 [刘建平Pinard scikit-learn随机森林调参小结](https://www.cnblogs.com/pinard/p/6160412.html)